## 심층 신경망

In [15]:
# 실행마다 동일한 결과를 얻기 위해 케라스에 랜덤 시드를 사용하고 텐서플로 연산을 결정적으로 만듦. 
import tensorflow as tf

tf.keras.utils.set_random_seed(42)
tf.config.experimental.enable_op_determinism()

### 2개의 층

In [16]:
from tensorflow import keras

(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

In [17]:
from sklearn.model_selection import train_test_split

train_scaled = train_input / 255.0 #이미지 픽셀값 정규화
train_scaled = train_scaled.reshape(-1, 28*28) #2차원 배열을 784크기의 1차원 배열로 변환

train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42)

입력층과 출력층 사이에 은닉층을 추가하고 항상 활성화 함수가 있음<br>
은닉층의 활성화 함수는 출력층보다 비교적 자유로운 편<br>
대표적으로 시그모이드 함수와 렐루함수 등을 사용함

In [18]:
#은닉층, 뉴런의 개수는 특별한 기준은 없음, 출력층의 뉴런보다는 많아야 함
dense1 = keras.layers.Dense(100, activation='sigmoid', input_shape=(784,))
#출력층, 10개의 클래스를 분류하므로 10개의 뉴런을 둠
dense2 = keras.layers.Dense(10, activation='softmax')

### 심층 신경망 만들기

In [19]:
model = keras.Sequential([dense1, dense2]) #순서 주의 (방법1)

In [20]:
#출력 순서는 은닉층->출력층
#None: fit()메서드에서 미니배치 경사 하강법을 사용하는데 샘플 개수를 고정하지 않고 유연하게 대응하기 위해
#784x100개+100개=78,500개
#100x10개+10개=1,010개
model.summary() 

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 100)               78500     
                                                                 
 dense_5 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


### 층을 추가하는 다른 방법

In [21]:
#Sequential 클래스의 생성자 안에서 바로 Dense 클래스의 객체를 만듦 (방법2)
model = keras.Sequential([
    keras.layers.Dense(100, activation='sigmoid', input_shape=(784,), name='hidden'),
    keras.layers.Dense(10, activation='softmax', name='output')
], name='패션 MNIST 모델')

In [22]:
model.summary()

Model: "패션 MNIST 모델"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden (Dense)              (None, 100)               78500     
                                                                 
 output (Dense)              (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [23]:
#add()메서드 호출 (방법3)
model = keras.Sequential()
model.add(keras.layers.Dense(100, activation='sigmoid', input_shape=(784,)))
model.add(keras.layers.Dense(10, activation='softmax'))

In [24]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 100)               78500     
                                                                 
 dense_7 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [25]:
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')

#추가된 층이 성능을 향상시켰다는 것을 알 수 있음
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 2s 981us/step - loss: 0.5628 - accuracy: 0.8073
Epoch 2/5
1500/1500 [==============================] - 1s 982us/step - loss: 0.4075 - accuracy: 0.8522
Epoch 3/5
1500/1500 [==============================] - 1s 983us/step - loss: 0.3741 - accuracy: 0.8652
Epoch 4/5
1500/1500 [==============================] - 1s 988us/step - loss: 0.3509 - accuracy: 0.8732
Epoch 5/5
1500/1500 [==============================] - 1s 983us/step - loss: 0.3335 - accuracy: 0.8784


### 렐루 활성화 함수

시그모이드 함수의 단점은 오른쪽과 왼쪽 끝으로 갈수록 그래프가 누워있어 올바른 출력을 신속하게 만들기가 어려움<br>
심층 신경망일수록 누적되어 학습을 더 어렵게 만듦 => 개선하여 **렐루(ReLU)함수**가 제안됨<br><br>
입력이 양수라면 값이 나오고 음수라면 0 (= $ max(0,z) $)

Flatten 클래스는 배치 차원을 제외하고 나머지 입력 차원을 모두 일렬로 펼치는 역할<br>
인공 신경망 성능을 위해 기여하는 바는 없지만 입력층과 은닉층 사이에 추가하기 때문에 층이라 부름

In [26]:
model=keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28,28))) #입력층 바로 뒤에 추가
model.add(keras.layers.Dense(100,activation='relu'))
model.add(keras.layers.Dense(10,activation='softmax'))

In [27]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense_8 (Dense)             (None, 100)               78500     
                                                                 
 dense_9 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [29]:
(train_input,train_target),(test_input,test_target)=keras.datasets.fashion_mnist.load_data()

train_scaled=train_input/255.0
train_scaled,val_scaled,train_target,val_target=train_test_split(train_scaled,train_target,test_size=0.2,random_state=42)

In [30]:
model.compile(loss='sparse_categorical_crossentropy',metrics='accuracy')
model.fit(train_scaled,train_target,epochs=5)

Epoch 1/5
1500/1500 [==============================] - 2s 976us/step - loss: 0.5336 - accuracy: 0.8090
Epoch 2/5
1500/1500 [==============================] - 1s 988us/step - loss: 0.3932 - accuracy: 0.8585
Epoch 3/5
1500/1500 [==============================] - 1s 991us/step - loss: 0.3566 - accuracy: 0.8718
Epoch 4/5
1500/1500 [==============================] - 2s 1000us/step - loss: 0.3340 - accuracy: 0.8807
Epoch 5/5
1500/1500 [==============================] - 1s 985us/step - loss: 0.3192 - accuracy: 0.8861


In [31]:
model.evaluate(val_scaled,val_target)

375/375 [==============================] - 0s 677us/step - loss: 0.3714 - accuracy: 0.8731


[0.3713562786579132, 0.8730833530426025]

In [38]:
import ex02

sn1=1-ex02.sn1()
#p1=1-ex02.p1()
t1=1-ex02.t1()

(28, 28)


In [39]:
import numpy as np

pred=model.predict(sn1.reshape(-1,28,28))
print(np.round(pred,decimals=0))
print(np.argmax(pred))

pred=model.predict(t1.reshape(-1,28,28))
print(np.round(pred,decimals=0))
print(np.argmax(pred))

1/1 [==============================] - 0s 55ms/step
[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]
8
1/1 [==============================] - 0s 13ms/step
[[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]]
6


### 옵티마이저

compile()메서드에서는 RMSprop을 사용, 케라스는 다양한 종류의 경사 하강법 알고리즘을 제공하며 이들을<br>
**옵티마이저**라고 부름<br>

In [40]:
model.compile(optimizer='sgd',loss='sparse_categorical_crossentropy',metrics='accuracy')

In [41]:
#위와 동일
sgd = keras.optimizers.SGD()
model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics='accuracy')

In [42]:
sgd=keras.optimizers.SGD(learning_rate=0.1)

![옵티마이저](https://user-images.githubusercontent.com/45377884/91630397-18838100-ea0c-11ea-8f90-515ef74599f1.png)

In [52]:
sgd = keras.optimizers.SGD(momentum=0.9, nesterov=True)

In [53]:
adagrad = keras.optimizers.Adagrad()
model.compile(optimizer=adagrad, loss='sparse_categorical_crossentropy', metrics='accuracy')

In [54]:
rmsprop = keras.optimizers.RMSprop()
model.compile(optimizer=rmsprop, loss='sparse_categorical_crossentropy', metrics='accuracy')

In [55]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [56]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')

model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 2s 840us/step - loss: 0.5240 - accuracy: 0.8167
Epoch 2/5
1500/1500 [==============================] - 1s 849us/step - loss: 0.3917 - accuracy: 0.8601
Epoch 3/5
1500/1500 [==============================] - 1s 849us/step - loss: 0.3535 - accuracy: 0.8723
Epoch 4/5
1500/1500 [==============================] - 1s 850us/step - loss: 0.3287 - accuracy: 0.8802
Epoch 5/5
1500/1500 [==============================] - 1s 846us/step - loss: 0.3069 - accuracy: 0.8868


In [57]:
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 0s 701us/step - loss: 0.3555 - accuracy: 0.8720


[0.3555358350276947, 0.871999979019165]

In [58]:
pred=model.predict(sn1.reshape(-1,28,28))
print(np.round(pred,decimals=0))
print(np.argmax(pred))

pred=model.predict(t1.reshape(-1,28,28))
print(np.round(pred,decimals=0))
print(np.argmax(pred))

1/1 [==============================] - 0s 31ms/step
[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]
8
1/1 [==============================] - 0s 13ms/step
[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]
8
